# Workshop Notebook 3: Observability Part 1 - Validation Rules

In the previous notebooks you uploaded the models and artifacts, then deployed the models to production through provisioning workspaces and pipelines. Now you're ready to put your feet up! But to keep your models operational, your work's not done once the model is in production. You must continue to monitor the behavior and performance of the model to insure that the model provides value to the business.

In this notebook, you will learn about adding validation rules to pipelines.

## Preliminaries

In the blocks below we will preload some required libraries.

For convenience, the following `helper functions` are defined to retrieve previously created workspaces, models, and pipelines:

* `get_workspace(name, client)`: This takes in the name and the Wallaroo client being used in this session, and returns the workspace matching `name`.  If no workspaces are found matching the name, raises a `KeyError` and returns `None`.
* `get_model_version(model_name, workspace)`: Retrieves the most recent model version from the model matching the `model_name` within the provided `workspace`.  If no model matches that name, raises a `KeyError` and returns `None`.
* `get_pipeline(pipeline_name, workspace)`: Retrieves the most pipeline from the workspace matching the `pipeline_name` within the provided `workspace`.  If no model matches that name, raises a `KeyError` and returns `None`.

In [2]:
# preload needed libraries 

import wallaroo
from wallaroo.object import EntityNotFoundError
from wallaroo.framework import Framework


from IPython.display import display

# used to display DataFrame information without truncating
from IPython.display import display
import pandas as pd
pd.set_option('display.max_colwidth', None)

import json
import datetime
import time

# used for unique connection names

import string
import random

In [3]:
## convenience functions from the previous notebooks
## these functions assume your connection to wallaroo is called wl

## convenience functions from the previous notebook

# return the workspace called <name> through the Wallaroo client.
def get_workspace(name, client):
    workspace = None
    for ws in client.list_workspaces():
        if ws.name() == name:
            workspace= ws
            return workspace
    # if no workspaces were found
    if workspace==None:
        raise KeyError(f"Workspace {name} was not found.")
    return workspace


# returns the most recent model version in a workspace for the matching `model_name`
def get_model_version(model_name, workspace):
    modellist = workspace.models()
    model_version = [m.versions()[-1] for m in modellist if m.name() == model_name]
    # if no models match, return None
    if len(modellist) <= 0:
        raise KeyError(f"Model {mname} not found in this workspace")
        return None
    return model_version[0]

# get a pipeline by name in the workspace
def get_pipeline(pipeline_name, workspace):
    plist = workspace.pipelines()
    pipeline = [p for p in plist if p.name() == pipeline_name]
    if len(pipeline) <= 0:
        raise KeyError(f"Pipeline {pipeline_name} not found in this workspace")
        return None
    return pipeline[0]


## Login to Wallaroo

Retrieve the previous workspace, model versions, and pipelines used in the previous notebook.

In [4]:
## blank space to log in 

wl = wallaroo.Client()

wallarooPrefix = "beautiful-platypus-3587."
wallarooSuffix = "wallaroo.community"

wl = wallaroo.Client(api_endpoint=f"https://{wallarooPrefix}api.{wallarooSuffix}", 
                    auth_endpoint=f"https://{wallarooPrefix}keycloak.{wallarooSuffix}", 
                    auth_type="sso")

# retrieve the previous workspace, model, and pipeline version

workspace_name = "workshop-finserv-john"

workspace = wl.get_workspace(name=workspace_name, create_if_not_exist=True)

# set your current workspace to the workspace that you just created
wl.set_current_workspace(workspace)

# optionally, examine your current workspace
wl.get_current_workspace()

model_name = 'classification-finserv-prime'

prime_model_version = wl.get_model(model_name)

pipeline_name = 'ccfraud-detector'

pipeline = wl.get_pipeline(pipeline_name)

display(workspace)
display(prime_model_version)
display(pipeline)


{'name': 'workshop-finserv-john', 'id': 304, 'archived': False, 'created_by': 'd1704c38-2016-4b1d-9407-85e7e6875e6d', 'created_at': '2023-09-14T14:19:43.052686+00:00', 'models': [{'name': 'classification-finserv-prime', 'versions': 1, 'owner_id': '""', 'last_update_time': datetime.datetime(2023, 9, 14, 14, 23, 5, 256418, tzinfo=tzutc()), 'created_at': datetime.datetime(2023, 9, 14, 14, 23, 5, 256418, tzinfo=tzutc())}, {'name': 'ccfraud-xgboost-version', 'versions': 1, 'owner_id': '""', 'last_update_time': datetime.datetime(2023, 9, 14, 14, 35, 36, 474265, tzinfo=tzutc()), 'created_at': datetime.datetime(2023, 9, 14, 14, 35, 36, 474265, tzinfo=tzutc())}], 'pipelines': [{'name': 'ccfraud-detector', 'create_time': datetime.datetime(2023, 9, 14, 14, 23, 33, 832605, tzinfo=tzutc()), 'definition': '[]'}]}

Name,classification-finserv-prime
Version,b677a370-1f85-4fb4-8f1a-febc3ee8ffa5
File Name,keras_ccfraud.onnx
SHA,bc85ce596945f876256f41515c7501c399fd97ebcb9ab3dd41bf03f8937b4507
Status,ready
Image Path,None
Updated At,2023-14-Sep 14:23:05


name,ccfraud-detector
created,2023-09-14 14:23:33.832605+00:00
last_updated,2023-09-14 15:31:58.261852+00:00
deployed,False
tags,
versions,"f0535f6b-a731-4496-b68d-71dbd22ac3ec, 8f5ac365-5338-4986-8e03-cd1360054ebc, f3e2febf-fb04-401d-a5ec-c1786f2325fe, c7cf9ad8-6ba3-40a6-9f7f-984f02722c45, 27769953-4ac0-408d-843a-54fbbb79a05a, f5cf42e5-ad8f-4bc6-ad1c-af6667a68e6a"
steps,classification-finserv-prime


## Deploy the Pipeline

Add the model version as a pipeline step to our pipeline, and deploy the pipeline.  You may want to check the pipeline steps to verify that the right model version is set for the pipeline step.

In [5]:
## blank space to get your pipeline and run a small batch of data through it to see the range of predictions

pipeline.clear()
pipeline.add_model_step(prime_model_version)

deploy_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(0.5).memory("1Gi").build()
pipeline.deploy(deployment_config=deploy_config)

multiple_result = pipeline.infer_from_file('../data/cc_data_1k.df.json')
display(multiple_result)


,time,in.tensor,out.dense_1,check_failures
0,2023-09-14 15:41:59.008,"[-1.0603297501, 2.3544967095, -3.5638788326, 5.1387348926, -1.2308457019, -0.7687824608, -3.5881228109, 1.8880837663, -3.2789674274, -3.9563254554, 4.0993439118, -5.6539176395, -0.8775733373, -9.131571192, -0.6093537873, -3.7480276773, -5.0309125017, -0.8748149526, 1.9870535692, 0.7005485718, 0.9204422758, -0.1041491809, 0.3229564351, -0.7418141657, 0.0384120159, 1.0993439146, 1.2603409756, -0.1466244739, -1.4463212439]",[0.99300325],0
1,2023-09-14 15:41:59.008,"[-1.0603297501, 2.3544967095, -3.5638788326, 5.1387348926, -1.2308457019, -0.7687824608, -3.5881228109, 1.8880837663, -3.2789674274, -3.9563254554, 4.0993439118, -5.6539176395, -0.8775733373, -9.131571192, -0.6093537873, -3.7480276773, -5.0309125017, -0.8748149526, 1.9870535692, 0.7005485718, 0.9204422758, -0.1041491809, 0.3229564351, -0.7418141657, 0.0384120159, 1.0993439146, 1.2603409756, -0.1466244739, -1.4463212439]",[0.99300325],0
2,2023-09-14 15:41:59.008,"[-1.0603297501, 2.3544967095, -3.5638788326, 5.1387348926, -1.2308457019, -0.7687824608, -3.5881228109, 1.8880837663, -3.2789674274, -3.9563254554, 4.0993439118, -5.6539176395, -0.8775733373, -9.131571192, -0.6093537873, -3.7480276773, -5.0309125017, -0.8748149526, 1.9870535692, 0.7005485718, 0.9204422758, -0.1041491809, 0.3229564351, -0.7418141657, 0.0384120159, 1.0993439146, 1.2603409756, -0.1466244739, -1.4463212439]",[0.99300325],0
3,2023-09-14 15:41:59.008,"[-1.0603297501, 2.3544967095, -3.5638788326, 5.1387348926, -1.2308457019, -0.7687824608, -3.5881228109, 1.8880837663, -3.2789674274, -3.9563254554, 4.0993439118, -5.6539176395, -0.8775733373, -9.131571192, -0.6093537873, -3.7480276773, -5.0309125017, -0.8748149526, 1.9870535692, 0.7005485718, 0.9204422758, -0.1041491809, 0.3229564351, -0.7418141657, 0.0384120159, 1.0993439146, 1.2603409756, -0.1466244739, -1.4463212439]",[0.99300325],0
4,2023-09-14 15:41:59.008,"[0.5817662108, 0.097881551, 0.1546819424, 0.4754101949, -0.1978862306, -0.4504344854, 0.0166540447, -0.0256070551, 0.0920561602, -0.2783917153, 0.0593299441, -0.0196585416, -0.4225083157, -0.1217538877, 1.5473094894, 0.2391622864, 0.3553974881, -0.7685165301, -0.7000849355, -0.1190043285, -0.3450517133, -1.1065114108, 0.2523411195, 0.0209441826, 0.2199267436, 0.2540689265, -0.0450225094, 0.1086773898, 0.2547179311]",[0.0010916889],0
...,...,...,...,...
996,2023-09-14 15:41:59.008,"[1.052355506, -0.7602601059, -0.3124601687, -0.5580714587, -0.6198353331, 0.6635428464, -1.2171685083, 0.3144529308, 0.2360632058, 0.878209955, -0.5518803042, -0.2781328417, -0.5675947058, -0.0982688053, 0.1475098349, -0.3097481612, -1.0898892231, 2.804466934, -0.4211447753, -0.7315488305, -0.5311840374, -0.9053830525, 0.5382443229, -0.68327623, -1.1848642272, 0.9872236995, -0.0260721428, -0.1405966468, 0.0759031399]",[0.00011596084],0
997,2023-09-14 15:41:59.008,"[-0.8464537996, -0.7608807925, 2.186072883, -0.1614362994, -0.4069378894, 0.734079177, -0.4611705734, 0.4751492626, 1.4952832213, -0.9349105827, -0.7654272171, 0.4362793613, -0.6623354486, -1.5326388376, -1.4311992842, -1.0573215483, 0.9304904478, -1.2836000946, -1.079419331, 0.7138847264, 0.2710369668, 1.1943291742, 0.2527110226, 0.3107779567, 0.4219366694, 2.4854295825, 0.1754876037, -0.2362979978, 0.9979986569]",[0.0002785325],0
998,2023-09-14 15:41:59.008,"[1.0046377125, 0.0343666504, -1.3512533246, 0.4160460291, 0.5910548281, -0.8187740907, 0.5840864966, -0.447623496, 1.1193896296, -0.1156579903, 0.1298919303, -2.6410683948, 1.1658091033, 2.3607999565, -0.4265055896, -0.4862102299, 0.5102253659, -0.3384745171, -0.4081285365, -0.199414607, 0.0151691668, 0.2644673476, -0.0483547565, 0.9869714364, 0.629627219, 0.8990505678, -0.3731273846, -0.2166148809, 0.6374669208]",[0.0011070371],0
999,2023-09-14 15:41:59.008,"[0.4951101913, -0.2499369449, 0.4553345161, 0.9242750451, -0.3643510229, 0.602688482, -0.3785553207, 0.3170957153, 0.7368986387, -0.1195106678, 0.4017042912, 0.7371143425, -1.222

## Model Validation Rules

A simple way to try to keep your model's behavior up to snuff is to make sure that it receives inputs that it expects, and that its output is something that downstream systems can handle. This can entail specifying rules that document what you expect, and either enforcing these rules (by refusing to make a prediction), or at least logging an alert that the expectations described by your validation rules have been violated. As the developer of the model, the data scientist (along with relevant subject matter experts) will often be the person in the best position to specify appropriate validation rules.

In our house price prediction example, suppose you know that house prices in your market are typically in the range $750,000 to $1.5M dollars. Then you might want to set validation rules on your model pipeline to specify that you expect the model's predictions to also be in that range. Then, if the model predicts a value outside that range, the pipeline will log that one of the validation checks has failed; this allows you to investigate that instance further.

Note that in this specific example, a model prediction outside the specified range may not necessarily be "wrong"; but out-of-range predictions are likely unusual enough that you may want to "sanity-check" the model's behavior in these situations.

Wallaroo has functionality for specifying simple validation rules on model input and output values.

```python
pipeline.add_validation(<rulename>, <expression>)

```

Here, `<rulename>` is the name of the rule, and `<expression>` is a simple logical expression that the data scientist **expects to be true**.  This means if the expression proves **false**, then a `check_failure` flag is set in the inference results.

To add a validation step to a simple one-step pipeline, you need a handle to the pipeline (here called `pipeline`), and a handle to the model in the pipeline (here called `model`).  Then you can specify an expected prediction range as follows:

* Get the pipeline.
* Depending on the steps, the pipeline can be cleared and the sample model added as a step.
* Add the validation.
* Deploy the pipeline to set the validation as part of its steps

```python
# get the existing pipeline (in your workspace)
pipeline = get_pipeline("pipeline")

# you also need a handle to the model in this single-step pipeline.
# here are two ways to do it:
#
# (1) If you know the name of the model, you can also just use the get_model() convenience function above.
# In this example, the model has been uploaded to wallaroo with the name "mymodel"

model = get_model("mymodel") 

# (2) To get the model without knowing its name (for a single-step pipeline)
model = pipeline.model_configs()[0].model()


# specify the bounds
hi_bnd = 1500000.0 # 1.5M

#
# some examples of validation rules
#

# (1)  validation rule: prediction should be < 1.5 million
pipeline = pipeline.add_validation("less than 1.5m", model.outputs[0][0] < hi_bnd)

# deploy the pipeline to set the steps
pipeline.deploy()

```

When data is passed to the pipeline for inferences, the pipeline will log a check failure whenever one of the validation expressions evaluates to false. Here are some examples of inference results from a pipeline with the validation rule `model.outputs[0][0] < 400000.0`.

![Inferences with check failures](./images/validation_results.png)

You can also find check failures in the logs:

```python
logs = pipeline.logs()
logs.loc[logs['check_failures'] > 0]
```

### Model Validation Rules Exercise

Add some simple validation rules to the model pipeline that you created in a previous exercise.

* Add an upper bound or a lower bound to the model predictions
* Try to create predictions that fall both in and out of the specified range
* Look through the logs to find the check failures.

**HINT 1**: since the purpose of this exercise is try out validation rules, it might be a good idea to take a small data set and make predictions on that data set first, *then* set the validation rules based on those predictions, so that you can see the check failures trigger.

Here's an example:

```python
hi_bnd = 600000.0

pipeline = pipeline.add_validation("less than 600k", prime_model_version.outputs[0][0] < hi_bnd)

pipeline.deploy()

multiple_result = pipeline.infer_from_file('../data/test_data.df.json')

display(multiple_result[multiple_result['check_failures'] > 0])
```

In [6]:
# blank space to set a validation rule on the pipeline and check if it triggers as expected

hi_bnd = 0.50

pipeline = pipeline.add_validation("greater than 0.50", prime_model_version.outputs[0][0] > hi_bnd)

pipeline.deploy()

multiple_result = pipeline.infer_from_file('../data/cc_data_1k.df.json')

display(multiple_result[multiple_result['check_failures'] > 0])

,time,in.tensor,out.dense_1,check_failures
4,2023-09-14 15:47:26.545,"[0.5817662108, 0.097881551, 0.1546819424, 0.4754101949, -0.1978862306, -0.4504344854, 0.0166540447, -0.0256070551, 0.0920561602, -0.2783917153, 0.0593299441, -0.0196585416, -0.4225083157, -0.1217538877, 1.5473094894, 0.2391622864, 0.3553974881, -0.7685165301, -0.7000849355, -0.1190043285, -0.3450517133, -1.1065114108, 0.2523411195, 0.0209441826, 0.2199267436, 0.2540689265, -0.0450225094, 0.1086773898, 0.2547179311]",[0.0010916889],1
5,2023-09-14 15:47:26.545,"[-0.7621273681, 0.8854701414, 0.5235808652, -0.8139743551, 0.3793240544, 0.1560653365, 0.5451299665, 0.0785927242, 0.4143968543, 0.4914052348, 0.0774391022, 1.050105026, 0.9901440217, -0.6142483131, -1.5260740653, 0.2053324703, -1.0185637854, 0.0490986919, 0.6964184879, 0.5948331722, -0.3934362922, -0.592249266, -0.3953093077, -1.3310427025, 0.6287441287, 0.8665525996, 0.7974673604, 1.1174342262, -0.6700716551]",[0.00047266483],1
6,2023-09-14 15:47:26.545,"[-0.2836830107, 0.2281341608, 1.0358808685, 1.0311413647, 0.6485053917, 0.6993339, 0.1827667194, 0.0989746212, -0.5734487733, 0.5928927597, 0.3085637362, 0.1533869918, -0.3628347923, -0.2865054499, -1.1380446536, -0.2207117601, -0.1206033931, -0.2325246936, 0.8675179232, -0.0081323034, -0.015330415, 0.4169237822, -0.4249025314, -0.9834451977, -1.1175903578, 2.1076701885, -0.3361950073, -0.3469573431, 0.019307669]",[0.00082170963],1
7,2023-09-14 15:47:26.545,"[1.0379636346, -0.152987302, -1.0912561862, -0.0033339828, 0.4804281836, 0.1120708475, 0.0233157709, 0.0009213038, 0.4021730182, 0.2120753712, -0.1462804223, 0.4424477027, -0.4641602117, 0.498425643, -0.8230270969, 0.3168388184, -0.9050440977, 0.0710365039, 1.1111388587, -0.2157914054, -0.373759129, -1.0330075347, 0.3144720913, -0.5109243112, -0.1685910498, 0.5918324406, -0.2231792825, -0.2287153377, -0.0868944762]",[0.0011294782],1
8,2023-09-14 15:47:26.545,"[0.1517283662, 0.6589966337, -0.3323713647, 0.7285871979, 0.6430271573, -0.0361051306, 0.2201530504, -1.4928731939, -0.5895806487, 0.2227251103, 0.4443729713, 0.8411555815, -0.241291302, 0.898682875, -0.9866307096, -0.8919301767, -0.0878875914, 0.1163332461, 1.1469566646, -0.5417470436, 2.2321360563, -0.1679271382, -0.8071223667, -0.6379226787, 1.9121889391, -0.5565545169, 0.6528273964, 0.8163897966, -0.2281615017]",[0.0018743575],1
...,...,...,...,...
996,2023-09-14 15:47:26.545,"[1.052355506, -0.7602601059, -0.3124601687, -0.5580714587, -0.6198353331, 0.6635428464, -1.2171685083, 0.3144529308, 0.2360632058, 0.878209955, -0.5518803042, -0.2781328417, -0.5675947058, -0.0982688053, 0.1475098349, -0.3097481612, -1.0898892231, 2.804466934, -0.4211447753, -0.7315488305, -0.5311840374, -0.9053830525, 0.5382443229, -0.68327623, -1.1848642272, 0.9872236995, -0.0260721428, -0.1405966468, 0.0759031399]",[0.00011596084],1
997,2023-09-14 15:47:26.545,"[-0.8464537996, -0.7608807925, 2.186072883, -0.1614362994, -0.4069378894, 0.734079177, -0.4611705734, 0.4751492626, 1.4952832213, -0.9349105827, -0.7654272171, 0.4362793613, -0.6623354486, -1.5326388376, -1.4311992842, -1.0573215483, 0.9304904478, -1.2836000946, -1.079419331, 0.7138847264, 0.2710369668, 1.1943291742, 0.2527110226, 0.3107779567, 0.4219366694, 2.4854295825, 0.1754876037, -0.2362979978, 0.9979986569]",[0.0002785325],1
998,2023-09-14 15:47:26.545,"[1.0046377125, 0.0343666504, -1.3512533246, 0.4160460291, 0.5910548281, -0.8187740907, 0.5840864966, -0.447623496, 1.1193896296, -0.1156579903, 0.1298919303, -2.6410683948, 1.1658091033, 2.3607999565, -0.4265055896, -0.4862102299, 0.5102253659, -0.3384745171, -0.4081285365, -0.199414607, 0.0151691668, 0.2644673476, -0.0483547565, 0.9869714364, 0.629627219, 0.8990505678, -0.3731273846, -0.2166148809, 0.6374669208]",[0.0011070371],1
999,2023-09-14 15:47:26.545,"[0.4951101913, -0.2499369449, 0.4553345161, 0.9242750451, -0.3643510229, 0.602688482, -0.3785553207, 0.3170957153, 0.7368986387, -0.1195106678, 0.4017042912, 0.7371143425, -1.2229791154, 0.00619

## Clean Up

At this point, if you are not continuing on to the next notebook, undeploy your pipeline to give the resources back to the environment.

In [7]:
## blank space to undeploy the pipeline

pipeline.undeploy()

name,ccfraud-detector
created,2023-09-14 14:23:33.832605+00:00
last_updated,2023-09-14 15:47:22.515726+00:00
deployed,False
tags,
versions,"ea2dc58a-ab79-4b4b-9640-e42e47d61a48, f187c234-e24a-4eee-885d-0bc5f008725c, f0535f6b-a731-4496-b68d-71dbd22ac3ec, 8f5ac365-5338-4986-8e03-cd1360054ebc, f3e2febf-fb04-401d-a5ec-c1786f2325fe, c7cf9ad8-6ba3-40a6-9f7f-984f02722c45, 27769953-4ac0-408d-843a-54fbbb79a05a, f5cf42e5-ad8f-4bc6-ad1c-af6667a68e6a"
steps,classification-finserv-prime


## Congratulations!

In this workshop you have

* Set a validation rule on your house price prediction pipeline.
* Detected model predictions that failed the validation rule.

In the next notebook, you will learn how to monitor the distribution of model outputs for drift away from expected behavior.